# kNN для произвольных метрик близости

In [1]:
import pandas as pd
import numpy as np

- player – имя игрока
- pos – роль игрока на поле
- g – количество матчей, в которых участвовал игрок
- gs – количество матчей, в которых игрок был на поле с самого начала
- pts – общее количество очков

pos:
- SF small forward
- C center
- PF power forward
- SG shooting guard
- PG point guard
- G guard
- F forward

In [2]:
data = pd.read_csv( 'nba_2013.csv' )
data.head()

,player,pos,age,bref_team_id,g,gs,mp,fg,fga,fg.,...,drb,trb,ast,stl,blk,tov,pf,pts,season,season_end
0,Quincy Acy,SF,23,TOT,63,0,847,66,141,0.468,...,144,216,28,23,26,30,122,171,2013-2014,2013
1,Steven Adams,C,20,OKC,81,20,1197,93,185,0.503,...,190,332,43,40,57,71,203,265,2013-2014,2013
2,Jeff Adrien,PF,27,TOT,53,12,961,143,275,0.520,...,204,306,38,24,36,39,108,362,2013-2014,2013
3,Arron Afflalo,SG,28,ORL,73,73,2552,464,1011,0.459,...,230,262,248,35,3,146,136,1330,2013-2014,2013
4,Alexis Ajinca,C,25,NOP,56,30,951,136,249,0.546,...,183,277,40,23,46,63,187,328,2013-2014,2013


Для построения модели возьмем несколько столбцов

In [3]:
predictionColumns = ['pos', 'age', 'g', 'gs']

In [4]:
dataForPrediction = data[ predictionColumns ]
dataForPrediction.head()

,pos,age,g,gs
0,SF,23,63,0
1,C,20,81,20
2,PF,27,53,12
3,SG,28,73,73
4,C,25,56,30


pos - категориальная переменная, надо перевести в количественную

In [5]:
dataForPrediction = pd.get_dummies( dataForPrediction )
dataForPrediction.head()

,age,g,gs,pos_C,pos_F,pos_G,pos_PF,pos_PG,pos_SF,pos_SG
0,23,63,0,0,0,0,0,0,1,0
1,20,81,20,1,0,0,0,0,0,0
2,27,53,12,0,0,0,1,0,0,0
3,28,73,73,0,0,0,0,0,0,1
4,25,56,30,1,0,0,0,0,0,0


Заметим, что названия колонок нашего dataframe изменились

In [6]:
predictionColumns = dataForPrediction.columns.values
predictionColumns

array(['age', 'g', 'gs', 'pos_C', 'pos_F', 'pos_G', 'pos_PF', 'pos_PG',
       'pos_SF', 'pos_SG'], dtype=object)

Проведем нормализацию данных

In [7]:
dataNormalized = ( dataForPrediction - dataForPrediction.mean() ) / dataForPrediction.std()
dataNormalized.head()

,age,g,gs,pos_C,pos_F,pos_G,pos_PF,pos_PG,pos_SF,pos_SG
0,-0.835906,0.384886,-0.862207,-0.479271,-0.045596,-0.045596,-0.498831,-0.462818,1.962285,-0.540742
1,-1.550487,1.095711,-0.187863,2.082165,-0.045596,-0.045596,-0.498831,-0.462818,-0.508550,-0.540742
2,0.116868,-0.010016,-0.457600,-0.479271,-0.045596,-0.045596,2.000520,-0.462818,-0.508550,-0.540742
3,0.355062,0.779789,1.599148,-0.479271,-0.045596,-0.045596,-0.498831,-0.462818,-0.508550,1.845467
4,-0.359519,0.108454,0.149309,2.082165,-0.045596,-0.045596,-0.498831,-0.462818,-0.508550,-0.540742


Проверим, что у всех столбцов среднее 0 и среднеквадратичное отклонение 1

In [8]:
dataNormalized.describe()

,age,g,gs,pos_C,pos_F,pos_G,pos_PF,pos_PG,pos_SF,pos_SG
count,4.810000e+02,4.810000e+02,4.810000e+02,4.810000e+02,4.810000e+02,4.810000e+02,4.810000e+02,4.810000e+02,4.810000e+02,4.810000e+02
mean,1.073293e-16,1.763431e-16,7.247610e-17,-7.386099e-18,-1.283623e-16,-1.712940e-16,9.324950e-17,5.306451e-16,-2.049643e-16,-9.648092e-17
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-1.788680e+00,-2.063509e+00,-8.622067e-01,-4.792708e-01,-4.559608e-02,-4.559608e-02,-4.988309e-01,-4.628177e-01,-5.085504e-01,-5.407417e-01
25%,-8.359060e-01,-8.393113e-01,-8.622067e-01,-4.792708e-01,-4.559608e-02,-4.559608e-02,-4.988309e-01,-4.628177e-01,-5.085504e-01,-5.407417e-01
50%,-1.213252e-01,3.059057e-01,-5.250348e-01,-4.792708e-01,-4.559608e-02,-4.559608e-02,-4.988309e-01,-4.628177e-01,-5.085504e-01,-5.407417e-01
75%,5.932556e-01,8.982593e-01,9.585214e-01,-4.792708e-01,-4.559608e-02,-4.559608e-02,-4.988309e-01,-4.628177e-01,-5.085504e-01,-5.407417e-01
max,2.975192e+00,1.174691e+00,1.902603e+00,2.082165e+00,2.188612e+01,2.188612e+01,2.000520e+00,2.156186e+00,1.962285e+00,1.845467e+00


Выберем игрока, для которого будем искать наиболее близких

In [9]:
selectedPlayer = dataNormalized.loc[ 0 ]
selectedPlayer

age      -0.835906
g         0.384886
gs       -0.862207
pos_C    -0.479271
pos_F    -0.045596
pos_G    -0.045596
pos_PF   -0.498831
pos_PG   -0.462818
pos_SF    1.962285
pos_SG   -0.540742
Name: 0, dtype: float64

In [10]:
def euclidean_distance( player, selectedPlayer ):
    """
    Считаем "расстояние" между выбранным игроком selected_player и текущим player.
    В нашем случае это обычное евклидово расстояние
    """
      
    dist = 0
    for metricName in predictionColumns:
        dist += ( player[ metricName ] - selectedPlayer[ metricName ] )**2
    
    return dist**0.5

In [11]:
def euclidean_distance2( x, y ):
    """
    Считаем "расстояние" между выбранным игроком selected_player и текущим player.
    Аналогичный расчет для использования в модели KNeighborsRegressor
    """
    
    return np.sum( (x-y)**2 )**0.5

Введем новый столбец "близости" игроков к выбранному

In [12]:
dataNormalized['distance'] = dataNormalized.apply( euclidean_distance, axis = 1, args = (selectedPlayer,) )
dataNormalized.head()

,age,g,gs,pos_C,pos_F,pos_G,pos_PF,pos_PG,pos_SF,pos_SG,distance
0,-0.835906,0.384886,-0.862207,-0.479271,-0.045596,-0.045596,-0.498831,-0.462818,1.962285,-0.540742,0.000000
1,-1.550487,1.095711,-0.187863,2.082165,-0.045596,-0.045596,-0.498831,-0.462818,-0.508550,-0.540742,3.759870
2,0.116868,-0.010016,-0.457600,-0.479271,-0.045596,-0.045596,2.000520,-0.462818,-0.508550,-0.540742,3.684999
3,0.355062,0.779789,1.599148,-0.479271,-0.045596,-0.045596,-0.498831,-0.462818,-0.508550,1.845467,4.408134
4,-0.359519,0.108454,0.149309,2.082165,-0.045596,-0.045596,-0.498831,-0.462818,-0.508550,-0.540742,3.740656


Объединяем вычисления с исходными данными

In [13]:
dataWithDistance = data.join( dataNormalized['distance'] )
dataWithDistance.head()

,player,pos,age,bref_team_id,g,gs,mp,fg,fga,fg.,...,trb,ast,stl,blk,tov,pf,pts,season,season_end,distance
0,Quincy Acy,SF,23,TOT,63,0,847,66,141,0.468,...,216,28,23,26,30,122,171,2013-2014,2013,0.000000
1,Steven Adams,C,20,OKC,81,20,1197,93,185,0.503,...,332,43,40,57,71,203,265,2013-2014,2013,3.759870
2,Jeff Adrien,PF,27,TOT,53,12,961,143,275,0.520,...,306,38,24,36,39,108,362,2013-2014,2013,3.684999
3,Arron Afflalo,SG,28,ORL,73,73,2552,464,1011,0.459,...,262,248,35,3,146,136,1330,2013-2014,2013,4.408134
4,Alexis Ajinca,C,25,NOP,56,30,951,136,249,0.546,...,277,40,23,46,63,187,328,2013-2014,2013,3.740656


In [14]:
dataWithDistance.sort_values( by = 'distance', ascending = True ).head(6)

,player,pos,age,bref_team_id,g,gs,mp,fg,fga,fg.,...,trb,ast,stl,blk,tov,pf,pts,season,season_end,distance
0,Quincy Acy,SF,23,TOT,63,0,847,66,141,0.468,...,216,28,23,26,30,122,171,2013-2014,2013,0.000000
181,Jordan Hamilton,SF,23,TOT,60,12,1019,147,376,0.391,...,194,52,44,20,46,67,404,2013-2014,2013,0.421594
213,Robbie Hummel,SF,24,MIN,53,5,655,67,177,0.379,...,132,23,16,2,10,57,181,2013-2014,2013,0.491025
22,Darrell Arthur,SF,25,DEN,68,1,1161,162,410,0.395,...,210,61,39,47,58,185,401,2013-2014,2013,0.516787
42,Michael Beasley,SF,25,MIA,55,2,831,177,355,0.499,...,172,42,23,21,57,93,436,2013-2014,2013,0.575586
106,Jae Crowder,SF,23,DAL,78,8,1254,130,296,0.439,...,194,60,59,21,41,100,356,2013-2014,2013,0.650877


Первую строчку не учитываем (это и был selected_player)

Считаем среднее для первых пяти игроков самых близких игроков

In [15]:
dataWithDistance.sort_values( by = 'distance', ascending = True ).iloc[1:6]['pts'].mean()

355.6

### То же самое с помощью KNeighborsRegressor

In [16]:
from sklearn.neighbors import KNeighborsRegressor

In [17]:
knn = KNeighborsRegressor( n_neighbors = 5 )

In [18]:
dataNormalized.iloc[1:, :-1].head()

,age,g,gs,pos_C,pos_F,pos_G,pos_PF,pos_PG,pos_SF,pos_SG
1,-1.550487,1.095711,-0.187863,2.082165,-0.045596,-0.045596,-0.498831,-0.462818,-0.50855,-0.540742
2,0.116868,-0.010016,-0.457600,-0.479271,-0.045596,-0.045596,2.000520,-0.462818,-0.50855,-0.540742
3,0.355062,0.779789,1.599148,-0.479271,-0.045596,-0.045596,-0.498831,-0.462818,-0.50855,1.845467
4,-0.359519,0.108454,0.149309,2.082165,-0.045596,-0.045596,-0.498831,-0.462818,-0.50855,-0.540742
5,-0.359519,-0.286448,-0.794772,2.082165,-0.045596,-0.045596,-0.498831,-0.462818,-0.50855,-0.540742


In [19]:
data.loc[1:, 'pts'].head()

1     265
2     362
3    1330
4     328
5      92
Name: pts, dtype: int64

In [20]:
knn.fit( dataNormalized.iloc[1:, :-1], data.loc[1:, 'pts'] )

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=5, p=2,
          weights='uniform')

Предсказание количества очков для selected_player (без крайнего правого столбца distance)

In [21]:
knn.predict( dataNormalized.iloc[:1, :-1] )

array([ 355.6])

### И еще раз с помощью KNeighborsRegressor, передавая метрику как функцию

In [22]:
knn = KNeighborsRegressor( n_neighbors = 5, metric = euclidean_distance2 )

In [23]:
knn.fit( dataNormalized.iloc[1:, :-1], data.loc[1:, 'pts'] )

KNeighborsRegressor(algorithm='auto', leaf_size=30,
          metric=<function euclidean_distance2 at 0x0000002636883C80>,
          metric_params=None, n_jobs=1, n_neighbors=5, p=2,
          weights='uniform')

In [24]:
knn.predict( dataNormalized.iloc[:1, :-1] )

array([ 355.6])